<a href="https://colab.research.google.com/github/rineuman/Detectron2MaskRCNN/blob/master/Detectron2MaskRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial can be found here: https://medium.com/@wendeehsu/instance-segmentation-with-detectron2-127fbe01b20b

In [4]:
# Mount Google Drive for saving trained models
import os 

from google.colab import drive
drive.mount('/content/drive')

DRIVE_DIR = '/content/drive/My Drive/trained_models'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8MB 25kB/s 
     |████████████████████████████████| 6.6MB 18.3MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101
     |████████████████████████████████| 276kB 4.9MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=d32cda024ef14f4bd444b7b5f887ccea1d0a14aa84d2b4f827b87c690227202b
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip

In [6]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
import os
import numpy as np
import json
import cv2
from detectron2.structures import BoxMode
print(torch.__version__, torch.cuda.is_available()) # 1.5.0+cu101 True
setup_logger()

1.5.0+cu101 True


<Logger detectron2 (DEBUG)>

Parsed via_export_json to appropriate detectron2 format

In [7]:
def get_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_export_json.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)
    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}
        
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = v["regions"]
        objs = []
        for anno in annos:
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]
            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
                "iscrowd": 0
                }
            objs.append(obj)
            print(objs)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

Register custom training set

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../")

In [8]:
!git clone https://github.com/rineuman/newDataSet.git

Cloning into 'newDataSet'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 104 (delta 10), reused 103 (delta 9), pack-reused 0
Receiving objects: 100% (104/104), 24.77 MiB | 32.43 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [15]:
from detectron2.data import DatasetCatalog, MetadataCatalog
path = "/content/newDataSet/" # path to your image folder
for d in ["train", "val"]:
    DatasetCatalog.register("barrier" + d, lambda d=d: get_dicts(path + "/" +  d))
    MetadataCatalog.get("barrier" + d).set(thing_classes=["barrier"])

Check if data can be loaded

In [14]:
import random
from google.colab.patches import cv2_imshow
from detectron2.utils.visualizer import Visualizer
dataset_dicts = get_dicts(path + "train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("mango_train"), scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

KeyError: ignored